# Rotation averaging example

This example script demonstrates how to solve rotation averaging problems using AutoTight.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from popr.examples import RotationLifter

lifter = RotationLifter(d=3, n_meas=3)

# solve the rotation averaging problem locally.
y = lifter.simulate_y(noise=1e-1)

theta_gt, *_ = lifter.local_solver(lifter.theta, y, verbose=False)
estimates = {"init gt": theta_gt}
for i in range(10):
    theta_init = lifter.sample_theta()
    theta_i, *_ = lifter.local_solver(theta_init, y, verbose=False)
    estimates[f"init random {i}"] = theta_i

fig, ax = lifter.plot(estimates=estimates)
ax.legend([])
plt.show(block=False)
print("done")

In [ ]:
# solve the rotation averaging with a naive SDP.
from popr.utils.plotting_tools import plot_matrix

Q = lifter.get_Q()
A_known = lifter.get_A_known()
constraints = lifter.get_A_b_list(lifter.get_A_known())

fig, axs = plt.subplots(1, len(A_known) + 1)
fig.set_size_inches(3*(len(A_known) + 1), 3)
for i in range(len(A_known)):
    plot_matrix(A_known[i].toarray(), ax=axs[i], title=f"A{i} ", colorbar=False)
plot_matrix(Q.toarray(), ax=axs[i+1], title="Q", colorbar=False)

In [ ]:
from cert_tools.sdp_solvers import solve_sdp
from cert_tools.linalg_tools import rank_project

X, info = solve_sdp(Q, constraints, verbose=False)

x, info_rank = rank_project(X, p=1)
print(f"EVR: {info_rank['EVR']:.2e}")

theta_opt = lifter.get_theta(x.flatten())

estimates = {"init gt": theta_gt, "SDP": theta_opt}
fig, ax = lifter.plot(estimates=estimates)